<a href="https://colab.research.google.com/github/weslleyalmeid/DataScience-Studies/blob/main/Tensorflow%202.0/06%20-%20Transfer%C3%AAncia%20de%20Aprendizado/Transfer%C3%AAncia_de_Aprendizado_e_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Etapa 1: Instalação das dependências

In [ ]:
!pip uninstall -y tensorflow #Comando necessário, pois o TensorFlow-gpu não desinstala a versão mais recente do Tensorflow, pode gerar conflitos.

In [ ]:
!pip install tensorflow-gpu==2.0

In [ ]:
!pip install tqdm

### Fazendo o download da base de dados de gatos e cachorros

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-04-20 00:55:41--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   131MB/s    in 0.5s    

2020-04-20 00:55:43 (131 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Etapa 2: Pré-processamento

### Importação das bibliotecas

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

### Descompactando a base de dados de gatos e cachorros

In [ ]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [ ]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [ ]:
zip_object.extractall("./")

In [ ]:
zip_object.close()

### Configurando os caminhos (paths)

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construindo o modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [ ]:
img_shape = (128, 128, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Congelando o modelo base

In [ ]:
base_model.trainable = False

### Definindo o cabeçalho personalizado da rede neural

In [ ]:
base_model.output

<tf.Tensor 'out_relu_2/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<tf.Tensor 'global_average_pooling2d_2/Identity:0' shape=(None, 1280) dtype=float32>

In [ ]:
prediction_layer = tf.keras.layers.Dense(units = 1, activation = "sigmoid")(global_average_layer)

### Definindo o modelo

In [ ]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

### Compilando o modelo

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001),
              loss="binary_crossentropy", metrics = ["accuracy"])

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: MobileNet (que estamos usando) suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Treinando o modelo

In [ ]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

16/16 [==============================] - 12s 753ms/step - loss: 0.8324 - accuracy: 0.5035 - val_loss: 0.9052 - val_accuracy: 0.4950
Epoch 2/5
16/16 [==============================] - 9s 590ms/step - loss: 0.7374 - accuracy: 0.5600 - val_loss: 0.8103 - val_accuracy: 0.5420
Epoch 3/5
16/16 [==============================] - 10s 598ms/step - loss: 0.6740 - accuracy: 0.6150 - val_loss: 0.7322 - val_accuracy: 0.5790
Epoch 4/5
16/16 [==============================] - 10s 604ms/step - loss: 0.6159 - accuracy: 0.6605 - val_loss: 0.6644 - val_accuracy: 0.6180
Epoch 5/5
16/16 [==============================] - 10s 595ms/step - loss: 0.5649 - accuracy: 0.7070 - val_loss: 0.6086 - val_accuracy: 0.6550


Epoch 1/5
16/16 [==============================] - 12s 735ms/step - loss: 0.8111 - accuracy: 0.4925 - val_loss: 0.8516 - val_accuracy: 0.4540
Epoch 2/5
16/16 [==============================] - 9s 580ms/step - loss: 0.7420 - accuracy: 0.5530 - val_loss: 0.7644 - val_accuracy: 0.5150
Epoch 3/5
16/16 [==============================] - 9s 577ms/step - loss: 0.6811 - accuracy: 0.6025 - val_loss: 0.6908 - val_accuracy: 0.5810
Epoch 4/5
16/16 [==============================] - 9s 585ms/step - loss: 0.6285 - accuracy: 0.6565 - val_loss: 0.6221 - val_accuracy: 0.6570
Epoch 5/5
16/16 [==============================] - 9s 587ms/step - loss: 0.5807 - accuracy: 0.6990 - val_loss: 0.5623 - val_accuracy: 0.7110


### Avaliação do modelo de transferência de aprendizagem

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [ ]:
valid_accuracy

0.655

0.711

## Fine tuning


Duas questões principais:

- NÃO USE Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 

### Descongelando algumas camadas do topo do modelo base

In [ ]:
base_model.trainable = True
len(base_model.layers)

155

155

In [ ]:
fine_tuning_at = 100

In [ ]:
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

### Compilando o modelo para fine tuning

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Fine tuning

In [ ]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 12s 772ms/step - loss: 0.1962 - accuracy: 0.9255 - val_loss: 0.1949 - val_accuracy: 0.9350
Epoch 2/5
16/16 [==============================] - 10s 619ms/step - loss: 0.0297 - accuracy: 0.9975 - val_loss: 0.2170 - val_accuracy: 0.9320
Epoch 3/5
16/16 [==============================] - 10s 623ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.1753 - val_accuracy: 0.9520
Epoch 4/5
16/16 [==============================] - 10s 611ms/step - loss: 0.0058 - accuracy: 0.9995 - val_loss: 0.2475 - val_accuracy: 0.9360
Epoch 5/5
16/16 [==============================] - 10s 623ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.2907 - val_accuracy: 0.9330


Epoch 1/5
16/16 [==============================] - 12s 763ms/step - loss: 0.1875 - accuracy: 0.9310 - val_loss: 0.1120 - val_accuracy: 0.9620
Epoch 2/5
16/16 [==============================] - 10s 626ms/step - loss: 0.0306 - accuracy: 0.9955 - val_loss: 0.1028 - val_accuracy: 0.9640
Epoch 3/5
16/16 [==============================] - 10s 630ms/step - loss: 0.0116 - accuracy: 0.9985 - val_loss: 0.1388 - val_accuracy: 0.9600
Epoch 4/5
16/16 [==============================] - 10s 619ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.2168 - val_accuracy: 0.9490
Epoch 5/5
16/16 [==============================] - 10s 621ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.2009 - val_accuracy: 0.9590


### Avaliação do modelo com fine tuning

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [ ]:
valid_accuracy

0.933

0.959